In [ ]:
import os
import sys
import threading
import time

import GCode
import GCode.Line
import GRBL
import matplotlib.pyplot as plt
import numpy as np

# Code:

Draw a 10 mm line from (0, 0) to (10, 0).

In [63]:
progs = list()

for n_lines in range(2, 11):

    line_cfg = {"X0": 0, "Xf": 10, "Y": 0, "n_points": 2}
    points = GCode.HorzLine(**line_cfg)

    line_cfg = {
        "points": points,
        "feed": 120,
        "power": 128,
        "dynamic_power": True,
    }
    line = GCode.Line(**line_cfg)

    prog_cfg = {"lines": [line] * n_lines, "feed": 120}
    prog = GCode.Program(**prog_cfg)
    progs.append(prog)
progs


[Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=3, dist=60.00mm, t=21.00s>,
 Program<lines=4, dist=80.00mm, t=28.00s>,
 Program<lines=5, dist=100.00mm, t=35.00s>,
 Program<lines=6, dist=120.00mm, t=42.00s>,
 Program<lines=7, dist=140.00mm, t=49.00s>,
 Program<lines=8, dist=160.00mm, t=56.00s>,
 Program<lines=9, dist=180.00mm, t=63.00s>,
 Program<lines=10, dist=200.00mm, t=70.00s>]

In [64]:
def concurrent_map(func, data):    """    Similar to the bultin function map(). But spawn a thread for each argument    and apply `func` concurrently.    Note: unlike map(), we cannot take an iterable argument. `data` should be an    indexable sequence.    """    N = len(data)    result = [None] * N    # wrapper to dispose the result in the right slot    def task_wrapper(i):        result[i] = func(data[i])    threads = [        threading.Thread(target=task_wrapper, args=(i,)) for i in range(N)    ]    for t in threads:        t.start()    for t in threads:        t.join()    return result

In [67]:
line_config = {"X0": 0, "Xf": 10, "Y": 0}
pointss = concurrent_map(
    lambda n: GCode.HorzLine(n_points=n, **line_config), range(2, 11)
)


In [70]:
line_cfg = {"feed": 120, "power": 128, "dynamic_power": True}
liness = concurrent_map(
    lambda points: GCode.Line(points=points, **line_cfg), pointss
)
liness


[Line<len=10.0mm, feed=120, power=128>,
 Line<len=10.0mm, feed=120, power=128>,
 Line<len=10.0mm, feed=120, power=128>,
 Line<len=10.0mm, feed=120, power=128>,
 Line<len=10.0mm, feed=120, power=128>,
 Line<len=10.0mm, feed=120, power=128>,
 Line<len=10.0mm, feed=120, power=128>,
 Line<len=10.0mm, feed=120, power=128>,
 Line<len=10.0mm, feed=120, power=128>]

In [58]:
concurrent_map(lambdap prog: prog.generate_gcode(), progs)

[None, None, None, None, None, None, None, None, None]

In [59]:
concurrent_map(lambda prog: len(prog.buffer), progs)


[17, 21, 25, 29, 33, 37, 41, 45, 49]

In [60]:
concurrent_map(lambda prog: prog.__repr__(), progs)


['Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=3, dist=60.00mm, t=21.00s>',
 'Program<lines=4, dist=80.00mm, t=28.00s>',
 'Program<lines=5, dist=100.00mm, t=35.00s>',
 'Program<lines=6, dist=120.00mm, t=42.00s>',
 'Program<lines=7, dist=140.00mm, t=49.00s>',
 'Program<lines=8, dist=160.00mm, t=56.00s>',
 'Program<lines=9, dist=180.00mm, t=63.00s>',
 'Program<lines=10, dist=190.00mm, t=65.00s>']

In [61]:
concurrent_map(lambda prog: prog.jog_dist, progs)


[20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 90.0]

In [49]:
concurrent_map(lambda prog: prog.laserin_dist, progs)


[20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]

In [50]:
for prog in progs:
    for line in prog.lines:
        break


In [51]:
list(map(lambda line: line.x_0, prog.lines))


[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

In [52]:
list(map(lambda line: line.x_f, prog.lines))


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [53]:
prog.jog_dist


90.0

In [54]:
for idx in range(len(prog.lines)):
    if np.mod(idx, 2) == 1:
        print(idx)
# s        prog.lines[idx].reverse()
prog.jog_dist


1
3
5
7
9


90.0